In [ ]:
####### USER INPUT
import mdtraj as md

top = '/Users/joshsmith/Git/plumitas/plumitas/data/topology/conf.gro'
traj = md.load(top)
topology = traj.top

table, bonds = topology.to_dataframe()
# sidechain_atoms = traj.topology.select('protein and not backbone')
# sidechain_heavy = table.iloc[sidechain_atoms, :][table['element'] != 'H']

plumed = {'header': {'restart': False,
                     'wholemolecules': ['protein', 'resname ALA']},

          'groups': {'sidechain_com': {'com': 'sidechain and resname ALA'}
                     },

          'collective_variables': {
              'phi': {'torsion': {'angle': 'phi',
                                  'resid': 2,
                                  'atoms': '',
                                  }
                      },
              'psi': {'torsion': {'atoms': '7,9,15,17',
                                  'angle': 'psi',
                                  'resid': 2,
                                  },
                      },
          },

          'bias': {'pbmetad': {'label': 'pbmetad',
                               'arg': 'phi,psi',
                               'temp': '300',
                               'pace': '500',
                               'biasfactor': '15',
                               'height': '1.2',
                               'sigma': '0.35,0.35',
                               'grid_min': '-pi,-pi',
                               'grid_max': 'pi,pi',
                               'grid_spacing': '0.1,0.1',
                               'file': 'HILLS_phi,HILLS_psi'
                               }
                   },

          'footer': {'print': {'stride': '500',
                               'arg': 'phi,psi,pbmetad.bias',
                               'file': 'COLVAR'
                               },
                     }
          }

generate_input(topology, **plumed)
